In [101]:
# importando as bibliotecas
import numpy as np
import pandas as pd

In [102]:
# carregar o dataset
df = pd.read_csv('data/diabetes.csv')
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75832 entries, 0 to 75831
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   encounter_id              75832 non-null  float64
 1   patient_nbr               75832 non-null  float64
 2   race                      75832 non-null  object 
 3   gender                    75832 non-null  object 
 4   age                       75832 non-null  object 
 5   weight                    75832 non-null  object 
 6   admission_type_id         75832 non-null  float64
 7   discharge_disposition_id  75832 non-null  float64
 8   admission_source_id       75832 non-null  float64
 9   time_in_hospital          75832 non-null  float64
 10  payer_code                75832 non-null  object 
 11  medical_specialty         75832 non-null  object 
 12  num_lab_procedures        75832 non-null  float64
 13  num_procedures            75832 non-null  float64
 14  num_me

(75832, 50)

In [103]:
# Realizando drop de colunas de identificação
df.drop(['diag_1', 'diag_2', 'diag_3', 'encounter_id','patient_nbr'], axis=1, inplace=True)

In [104]:
for col in df.columns:
    print(f'{col}: {df[col].unique()}\n')

race: ['Caucasian' 'AfricanAmerican' '?' 'Other' 'Asian' 'Hispanic']

gender: ['Female' 'Male' 'Unknown/Invalid']

age: ['[0-10)' '[10-20)' '[20-30)' '[30-40)' '[40-50)' '[50-60)' '[60-70)'
 '[70-80)' '[80-90)' '[90-100)']

weight: ['?' '[75-100)' '[50-75)' '[0-25)' '[100-125)' '[25-50)' '[125-150)'
 '[175-200)' '[150-175)' '>200']

admission_type_id: [6. 1. 2. 3. 4. 5. 8. 7.]

discharge_disposition_id: [25.  1.  3.  6.  2.  5. 11.  7. 10.  4. 14. 18.  8. 13. 12. 16. 17. 22.
 23.  9. 20. 15. 24. 28. 19. 27.]

admission_source_id: [ 1.  7.  2.  4.  5.  6. 20.  3. 17.  8.  9. 14. 10. 22.]

time_in_hospital: [ 1.  3.  2.  4.  5. 13. 12.  9.  7. 10.  6. 11.  8. 14.]

payer_code: ['?' 'MC' 'MD' 'HM' 'UN' 'BC' 'SP' 'CP' 'SI' 'DM' 'CM' 'CH' 'PO' 'WC' 'OT'
 'OG' 'MP']

medical_specialty: ['Pediatrics-Endocrinology' '?' 'InternalMedicine'
 'Family/GeneralPractice' 'Cardiology' 'Surgery-General' 'Orthopedics'
 'Gastroenterology' 'Surgery-Cardiovascular/Thoracic' 'Nephrology'
 'Orthopedics-Recons

In [105]:
for col in df.columns:
    print(f'{col}: {df[col].value_counts()}\n')

race: race
Caucasian          55548
AfricanAmerican    16003
?                   1511
Hispanic            1431
Other                958
Asian                381
Name: count, dtype: int64

gender: gender
Female             40950
Male               34880
Unknown/Invalid        2
Name: count, dtype: int64

age: age
[70-80)     19571
[60-70)     16599
[50-60)     13075
[80-90)     12249
[40-50)      7442
[30-40)      2949
[90-100)     1937
[20-30)      1271
[10-20)       586
[0-10)        153
Name: count, dtype: int64

weight: weight
?            73317
[75-100)      1061
[50-75)        724
[100-125)      465
[125-150)      107
[25-50)         77
[0-25)          45
[150-175)       26
[175-200)        8
>200             2
Name: count, dtype: int64

admission_type_id: admission_type_id
1.0    38300
2.0    14437
3.0    13422
5.0     4742
6.0     4677
8.0      237
7.0       10
4.0        7
Name: count, dtype: int64

discharge_disposition_id: discharge_disposition_id
1.0     44586
3.0      9758


In [106]:
# substitua os valores ? por np.nan
df = df.replace('?', np.nan)

# Printa valores iguais a nulo nas colunas e seus respectivos percentuais em relação ao total
for col in df.columns:
    if df[col].isnull().sum() > 0:
        print(f'{col}: {df[col].isnull().sum()}  ({df[col].isnull().mean() * 100:.2f}%) valores nulos')

race: 1511  (1.99%) valores nulos
weight: 73317  (96.68%) valores nulos
payer_code: 37437  (49.37%) valores nulos
medical_specialty: 32475  (42.82%) valores nulos
max_glu_serum: 70842  (93.42%) valores nulos
A1Cresult: 63534  (83.78%) valores nulos


In [107]:
# Realizando drop de colunas que possuem mais de 50% de valores nulos
df.drop(['weight', 'medical_specialty', 'max_glu_serum', 'payer_code', 'A1Cresult'], axis=1, inplace=True)

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75832 entries, 0 to 75831
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   race                      74321 non-null  object 
 1   gender                    75832 non-null  object 
 2   age                       75832 non-null  object 
 3   admission_type_id         75832 non-null  float64
 4   discharge_disposition_id  75832 non-null  float64
 5   admission_source_id       75832 non-null  float64
 6   time_in_hospital          75832 non-null  float64
 7   num_lab_procedures        75832 non-null  float64
 8   num_procedures            75832 non-null  float64
 9   num_medications           75832 non-null  float64
 10  number_outpatient         75832 non-null  float64
 11  number_emergency          75832 non-null  float64
 12  number_inpatient          75832 non-null  float64
 13  number_diagnoses          75832 non-null  float64
 14  metfor

In [109]:
df.fillna({'race': 'Other'}, inplace=True)
df['race'].unique()

array(['Caucasian', 'AfricanAmerican', 'Other', 'Asian', 'Hispanic'],
      dtype=object)

In [110]:
# exceto insuline 
medications = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide',
                'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol',
                'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'glyburide.metformin',
                'glipizide.metformin', 'glimepiride.pioglitazone', 'metformin.rosiglitazone',
                'metformin.pioglitazone']
# mostre a quantidade de cada categoria de cada coluna
for col in medications:
    print(f'{col}: {df[col].value_counts()}\n')


metformin: metformin
No        61427
Steady    13177
Up          808
Down        420
Name: count, dtype: int64

repaglinide: repaglinide
No        74745
Steady      974
Up           78
Down         35
Name: count, dtype: int64

nateglinide: nateglinide
No        75346
Steady      469
Up           13
Down          4
Name: count, dtype: int64

chlorpropamide: chlorpropamide
No        75753
Steady       72
Up            6
Down          1
Name: count, dtype: int64

glimepiride: glimepiride
No        72166
Steady     3278
Up          247
Down        141
Name: count, dtype: int64

acetohexamide: acetohexamide
No        75831
Steady        1
Name: count, dtype: int64

glipizide: glipizide
No        66186
Steady     8579
Up          632
Down        435
Name: count, dtype: int64

glyburide: glyburide
No        67316
Steady     7372
Up          691
Down        453
Name: count, dtype: int64

tolbutamide: tolbutamide
No        75812
Steady       20
Name: count, dtype: int64

pioglitazone: pioglita

In [111]:
# Para cada coluna de medicmentos, substitua os valores 'No' por 0 e 'Steady', 'Up' e 'Down' por 1
for col in medications:
    df[col] = np.where(df[col] == 'No', 0, 1)

for col in medications:
    print(f'{col}: {df[col].value_counts()}\n')


metformin: metformin
0    61427
1    14405
Name: count, dtype: int64

repaglinide: repaglinide
0    74745
1     1087
Name: count, dtype: int64

nateglinide: nateglinide
0    75346
1      486
Name: count, dtype: int64

chlorpropamide: chlorpropamide
0    75753
1       79
Name: count, dtype: int64

glimepiride: glimepiride
0    72166
1     3666
Name: count, dtype: int64

acetohexamide: acetohexamide
0    75831
1        1
Name: count, dtype: int64

glipizide: glipizide
0    66186
1     9646
Name: count, dtype: int64

glyburide: glyburide
0    67316
1     8516
Name: count, dtype: int64

tolbutamide: tolbutamide
0    75812
1       20
Name: count, dtype: int64

pioglitazone: pioglitazone
0    70674
1     5158
Name: count, dtype: int64

rosiglitazone: rosiglitazone
0    70403
1     5429
Name: count, dtype: int64

acarbose: acarbose
0    75603
1      229
Name: count, dtype: int64

miglitol: miglitol
0    75804
1       28
Name: count, dtype: int64

troglitazone: troglitazone
0    75829
1       

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75832 entries, 0 to 75831
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   race                      75832 non-null  object 
 1   gender                    75832 non-null  object 
 2   age                       75832 non-null  object 
 3   admission_type_id         75832 non-null  float64
 4   discharge_disposition_id  75832 non-null  float64
 5   admission_source_id       75832 non-null  float64
 6   time_in_hospital          75832 non-null  float64
 7   num_lab_procedures        75832 non-null  float64
 8   num_procedures            75832 non-null  float64
 9   num_medications           75832 non-null  float64
 10  number_outpatient         75832 non-null  float64
 11  number_emergency          75832 non-null  float64
 12  number_inpatient          75832 non-null  float64
 13  number_diagnoses          75832 non-null  float64
 14  metfor

In [113]:
# Binazirando a coluna change
df['change'] = np.where(df['change'] == 'No', 0, 1)
df['change'].value_counts()


change
0    42275
1    33557
Name: count, dtype: int64

In [114]:

# Binazirando a coluna diabetesMed
df['diabetesMed'] = np.where(df['diabetesMed'] == 'No', 0, 1)
df['diabetesMed'].value_counts()

diabetesMed
1    57641
0    18191
Name: count, dtype: int64